<a href="https://colab.research.google.com/github/jetsonmom/6.23_automobility_lesson/blob/main/%EC%88%98%EC%97%85_3_%ED%8C%8C%EC%9D%B4%EC%84%A0_list_%26dictionary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. 센서 데이터 필터링 - 안전 거리 이상의 장애물만 추출
sensor_distances = [2.5, 8.3, 1.2, 15.7, 3.8, 0.9, 12.4, 6.1]
safe_distances = [dist for dist in sensor_distances if dist > 5.0]
print("안전 거리 이상 장애물:", safe_distances)

In [ ]:
# 2. 좌표 변환 - 상대 좌표를 절대 좌표로 변환
vehicle_position = (10, 20)  # 차량 현재 위치
relative_points = [(1, 2), (-3, 4), (5, -1), (0, 3)]
absolute_points = [(x + vehicle_position[0], y + vehicle_position[1])
                   for x, y in relative_points]
print("절대 좌표:", absolute_points)

In [ ]:
# 3. 속도 제한 적용 - 각 구간별 최대 속도 제한
current_speeds = [45, 75, 38, 82, 55, 95, 28]
speed_limits = [50, 70, 40, 80, 60, 90, 30]
adjusted_speeds = [min(current, limit)
                   for current, limit in zip(current_speeds, speed_limits)]
print("속도 제한 적용 후:", adjusted_speeds)

In [ ]:
# 4. 경로 포인트 생성 - 시작점과 끝점 사이의 중간 포인트들 생성
start_point = (0, 0)
end_point = (10, 8)
num_points = 5
path_points = [(start_point[0] + i * (end_point[0] - start_point[0]) / (num_points - 1),
                start_point[1] + i * (end_point[1] - start_point[1]) / (num_points - 1))
               for i in range(num_points)]
print("경로 포인트들:", path_points)

이 코드는 **여러 센서 중에서 위험한 상황을 감지한 센서들만 골라내는** 코드입니다:

## 센서 데이터 분석

### sensor_data 각 센서의 의미
```python
{'type': 'lidar', 'distance': 3.2, 'angle': 45}    # 라이다: 우측 45도, 3.2m
{'type': 'camera', 'distance': 8.5, 'angle': 0}    # 카메라: 정면, 8.5m  
{'type': 'radar', 'distance': 1.8, 'angle': -30}   # 레이더: 좌측 30도, 1.8m
{'type': 'lidar', 'distance': 12.3, 'angle': 90}   # 라이다: 우측 90도, 12.3m
```

## 위험 조건 설정
```python
sensor['distance'] < 5.0 and abs(sensor['angle']) < 60
```

**두 조건을 모두 만족해야 위험**:
1. `distance < 5.0`: **5미터 이내**에 장애물
2. `abs(angle) < 60`: **정면 기준 좌우 60도 이내** (전방 주시 범위)

## 각 센서별 검사 과정

### 1번 센서 (라이다)
- 거리: 3.2 < 5.0 ✅ (가까움)
- 각도: |45| < 60 ✅ (전방 범위)
- **결과**: **위험** ⚠️

### 2번 센서 (카메라)  
- 거리: 8.5 < 5.0 ❌ (충분히 멀음)
- **결과**: 안전

### 3번 센서 (레이더)
- 거리: 1.8 < 5.0 ✅ (매우 가까움)  
- 각도: |-30| < 60 ✅ (전방 범위)
- **결과**: **위험** ⚠️

### 4번 센서 (라이다)
- 거리: 12.3 < 5.0 ❌ (충분히 멀음)
- **결과**: 안전

## 최종 결과
```python
danger_sensors = [
    {'type': 'lidar', 'distance': 3.2, 'angle': 45},
    {'type': 'radar', 'distance': 1.8, 'angle': -30}
]
```

## 실제 의미
**"전방 60도 범위 내에서 5미터 이내에 장애물을 감지한 센서 2개 발견!"**

**자율주행차의 대응**:
- 즉시 감속 또는 정지
- 스티어링으로 회피
- 경고음 발생
- 비상등 점멸

**쉽게 말하면**: 앞쪽에서 가까운 거리에 뭔가 있다는 **위험 신호**를 보낸 센서들만 골라낸 것입니다!

In [ ]:
# 5. 보너스: 조건부 다중 필터링 - 위험 상황 감지
sensor_data = [
    {'type': 'lidar', 'distance': 3.2, 'angle': 45},
    {'type': 'camera', 'distance': 8.5, 'angle': 0},
    {'type': 'radar', 'distance': 1.8, 'angle': -30},
    {'type': 'lidar', 'distance': 12.3, 'angle': 90}
]

danger_sensors = [sensor for sensor in sensor_data
                  if sensor['distance'] < 5.0 and abs(sensor['angle']) < 60]
print("위험 감지 센서:", danger_sensors)

In [ ]:
# 1. 센서 데이터 관리 - 리스트와 딕셔너리만 사용
sensors = ['camera', 'lidar', 'radar', 'gps']
distances = [8.5, 12.3, 15.7, 0]  # GPS는 거리가 아니므로 0

# 센서별 감지 거리 매핑
sensor_data = {}
sensor_data['camera'] = 8.5
sensor_data['lidar'] = 12.3
sensor_data['radar'] = 15.7
sensor_data['gps'] = 0

print("센서 데이터:", sensor_data)

# 2. 경로 계획 - 목적지별 거리 정보
destinations = ['집', '회사', '마트', '주유소']
distances_km = [0, 15, 8, 12]

route_plan = {}
route_plan['집'] = 0
route_plan['회사'] = 15
route_plan['마트'] = 8
route_plan['주유소'] = 12

print("목적지별 거리:", route_plan)

# 3. 차량 상태 체크
vehicle_parts = ['엔진', '브레이크', '타이어', '배터리']
status_codes = ['정상', '정상', '교체필요', '정상']

vehicle_status = {}
vehicle_status['엔진'] = '정상'
vehicle_status['브레이크'] = '정상'
vehicle_status['타이어'] = '교체필요'
vehicle_status['배터리'] = '정상'

print("차량 상태:", vehicle_status)

# 4. 신호등 정보 관리
intersections = ['서울역', '강남역', '홍대입구']
light_colors = ['빨강', '초록', '노랑']

traffic_lights = {}
traffic_lights['서울역'] = '빨강'
traffic_lights['강남역'] = '초록'
traffic_lights['홍대입구'] = '노랑'

print("교차로 신호등:", traffic_lights)

# 5. 주차장 정보
parking_zones = ['A구역', 'B구역', 'C구역', 'D구역']
empty_spots = [5, 0, 12, 3]

parking_info = {}
parking_info['A구역'] = 5
parking_info['B구역'] = 0
parking_info['C구역'] = 12
parking_info['D구역'] = 3

print("주차장 빈자리:", parking_info)

# 6. 속도 제한 구간
road_sections = ['시내도로', '국도', '고속도로', '스쿨존']
speed_limits = [50, 70, 100, 30]

speed_zones = {}
speed_zones['시내도로'] = 50
speed_zones['국도'] = 70
speed_zones['고속도로'] = 100
speed_zones['스쿨존'] = 30

print("구간별 제한속도:", speed_zones)

# 7. 연료/충전소 정보
station_names = ['GS칼텍스', '현대오일뱅크', '테슬라충전소', 'SK에너지']
distances_to_stations = [2.5, 5.8, 1.2, 4.3]

fuel_stations = {}
fuel_stations['GS칼텍스'] = 2.5
fuel_stations['현대오일뱅크'] = 5.8
fuel_stations['테슬라충전소'] = 1.2
fuel_stations['SK에너지'] = 4.3

print("주유소까지 거리:", fuel_stations)

# 8. 날씨별 주행 모드
weather_conditions = ['맑음', '비', '눈', '안개']
drive_modes = ['일반', '주의', '서행', '서행']

weather_driving = {}
weather_driving['맑음'] = '일반'
weather_driving['비'] = '주의'
weather_driving['눈'] = '서행'
weather_driving['안개'] = '서행'

print("날씨별 주행모드:", weather_driving)

# 9. 간단한 데이터 조회 예제
print("\n=== 데이터 조회 예제 ===")
print("라이다 센서 거리:", sensor_data['lidar'], "미터")
print("회사까지 거리:", route_plan['회사'], "km")
print("타이어 상태:", vehicle_status['타이어'])
print("강남역 신호등:", traffic_lights['강남역'])
print("C구역 빈자리:", parking_info['C구역'], "개")
print("고속도로 제한속도:", speed_zones['고속도로'], "km/h")
print("가장 가까운 충전소:", "테슬라충전소", fuel_stations['테슬라충전소'], "km")
print("비올 때 주행모드:", weather_driving['비'])

미션  강남역 갈 때 강남대로, 테헤란로 중 선택하고 여의도 갈 때 올림픽대로, 한강대로 중 하나를 선택하는 코드 만들기 if문 추가